In [ ]:
import glob
import numpy as np
import sys
sys.path.append("..")
import os

import matplotlib.pyplot as plt
from IPython.display import HTML
from scipy.ndimage import gaussian_filter

# from scipy.cluster.vq import kmeans, whiten
# from scipy.ndimage import gaussian_filter, distance_transform_edt
# from scipy.signal import convolve
# from scipy.optimize import curve_fit
# from skimage import measure, transform, filters, morphology
# import pyvista as pv

# from numba import njit

from schiller_lab_tools.lb3d import read_hdf5, read_asc
from schiller_lab_tools.microstructure_analysis import fill, label_regions_hk
from schiller_lab_tools.visualization import write_vti

import time

In [ ]:
# data_path = "../../../p1-synthesis_of_magnetic_particle_bijels/raw_data/constant_vol/phi_0.15/Bx_0-By_0-Bz_0/Rp_7.9-Ro_7.9/"
data_path = "."

# if os.path.exists(data_path):
wd = read_hdf5(glob.glob(f"{data_path}/wd*.h5")[-1])
od = read_hdf5(glob.glob(f"{data_path}/od*.h5")[-1])
md, t = read_asc(glob.glob(f"{data_path}/md-cfg*.asc")[-1])
md = md.sort_values(by = ['p_id'])
md = md.reset_index()
md = md.drop(['index'], axis = 1)

fill_od = fill(od)
fill_wd = fill(wd)

# bijel = od - wd
bijel = (fill_od - fill_wd)/(fill_od + fill_wd)
bijel = gaussian_filter(bijel, sigma = 0.4)
# bijel = transform.downscale_local_mean(bijel, factors = (2, 2, 2))

boxDims = bijel.shape
slc = np.s_[boxDims[0]//2, :, :]
plt.imshow(bijel[slc], cmap = "bwr")
plt.colorbar()

# from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111, projection='3d')

# # voxelarray = np.where(phi_fill > 1, 1, 0)
# plot = ax.voxels(phi_fill)

write_vti(".", "phi_compare", {"phi":od - wd, "phi_fill":bijel})

# Helper functions

In [ ]:
def radial_distribution_function(L, positions, positions2 = None, binsize = None):
    r_max = np.ceil(np.sqrt(3)*L)
    Vsphere = 4/3*np.pi*(r_max**3)

    if binsize is None:
        bins = np.linspace(0, r_max, int(r_max) + 1) 
        slc = slice(0, int(np.ceil(np.sqrt(3)/4*L)))
    else:
        bins = np.arange(0, r_max, binsize)
        slc = slice(0, int(np.ceil(np.sqrt(3)/4*L)//binsize))

    if positions2 is None:
        N = positions.shape[0]/Vsphere
        delta = positions[:, np.newaxis] - positions
    else:
        N = (positions.shape[0] + positions2.shape[0])/Vsphere
        delta = positions[:, np.newaxis] - positions2
    
    delta = np.where(delta > L//2, delta - L, delta)
    euclid_dist = np.linalg.norm(delta, axis = -1)
    euclid_dist = np.unique(euclid_dist)[1:]
    
    hist, bin_edges = np.histogram(euclid_dist, bins = bins)
    volume_slice = 4/3*np.pi*(np.power(bin_edges[1:], 3) - np.power(bin_edges[:-1], 3))

    x = bin_edges[slc]
    y = hist/(N*volume_slice)
    y = y[slc]
    y /= y[-1]

    return x, y

In [ ]:
import numpy as np
from scipy.optimize import minimize

def ellipsoid_surface_constraint(x, center, R, Q):
    """ Ensures that a point x lies on the ellipsoid defined by center, R (radii), and Q (rotation). """
    x_local = np.linalg.inv(Q) @ (x - center)  # Transform to local ellipsoid coordinates
    return (x_local[0] / R[0])**2 + (x_local[1] / R[1])**2 + (x_local[2] / R[2])**2 - 1

def rotation_matrix_from_vector(v):
    """ Constructs a rotation matrix from an orientation unit vector v. """
    v = v / np.linalg.norm(v)
    # Create an arbitrary perpendicular vector
    if np.abs(v[0]) < 0.9:
        perp = np.array([1, 0, 0])
    else:
        perp = np.array([0, 1, 0])
    
    v1 = np.cross(v, perp)
    v1 /= np.linalg.norm(v1)
    v2 = np.cross(v, v1)
    return np.column_stack([v1, v2, v])

def closest_ellipsoid_distance(center1, orientation1, Rp1, Ro1, center2, orientation2, Rp2, Ro2):
    """ Finds the closest distance between two ellipsoids. """
    
    # Define radii and rotation matrices for each ellipsoid
    R1 = np.array([Rp1, Ro1, Ro1])
    R2 = np.array([Rp2, Ro2, Ro2])
    
    Q1 = rotation_matrix_from_vector(orientation1)
    Q2 = rotation_matrix_from_vector(orientation2)
    
    # Initial guess: linearly interpolate along the center-to-center vector
    direction = center2 - center1
    direction /= np.linalg.norm(direction)
    x1_guess = center1 + R1[0] * direction  # Approximate surface point on E1
    x2_guess = center2 - R2[0] * direction  # Approximate surface point on E2

    # Optimization: minimize the distance between x1 and x2
    def objective(vars):
        x1, x2 = vars[:3], vars[3:]
        return np.sum((x2 - x1) ** 2)  # Squared Euclidean distance

    # Constraints: points must remain on the ellipsoid surfaces
    constraints = [
        {'type': 'eq', 'fun': lambda vars: ellipsoid_surface_constraint(vars[:3], center1, R1, Q1)},
        {'type': 'eq', 'fun': lambda vars: ellipsoid_surface_constraint(vars[3:], center2, R2, Q2)}
    ]

    # Run optimization
    result = minimize(objective, np.concatenate([x1_guess, x2_guess]), constraints=constraints, method='SLSQP')
    
    if result.success:
        x1_opt, x2_opt = result.x[:3], result.x[3:]
        min_distance = np.linalg.norm(x2_opt - x1_opt)
        return min_distance, x1_opt, x2_opt
    else:
        raise RuntimeError("Optimization failed to converge.")

In [ ]:
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# array = phi_fill

# verts, faces, normals, values = measure.marching_cubes(array, level = 0, step_size = 8, allow_degenerate=False)

# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111, projection='3d')

# mesh = Poly3DCollection(verts[faces])
# # mesh.set_edgecolor('k')
# ax.add_collection3d(mesh)

## Distance between particles

In [ ]:
Rp1 = 12.6
Rp2 = Rp1
Ro1 = 6.3
Ro2 = Ro1

In [ ]:
p1 = md.iloc[1] # 13, 1215, 1019
p2 = md.iloc[695] # 16, 1216, 1021

center1 = p1[['x', 'y', 'z']].to_numpy()
orientation1 = p1[['o_x', 'o_y', 'o_z']].to_numpy()

center2 = p2[['x', 'y', 'z']].to_numpy()
orientation2 = p2[['o_x', 'o_y', 'o_z']].to_numpy()

d, _, _= closest_ellipsoid_distance(center1, orientation1, Rp1, Ro1, center2, orientation2, Rp2, Ro2)
d

In [ ]:
npart = md.shape[0]

dists = np.zeros((npart, npart))

for i in range(npart-1):
    for j in range(i+1, npart):

        p1 = md.iloc[i] # 13, 1215, 1019
        p2 = md.iloc[j] # 16, 1216, 1021

        center1 = p1[['x', 'y', 'z']].to_numpy()
        orientation1 = p1[['o_x', 'o_y', 'o_z']].to_numpy()

        center2 = p2[['x', 'y', 'z']].to_numpy()
        orientation2 = p2[['o_x', 'o_y', 'o_z']].to_numpy()

        dists[i, j], _, _= closest_ellipsoid_distance(center1, orientation1, Rp1, Ro1, center2, orientation2, Rp2, Ro2)

# Visualization

In [ ]:
# x = np.random.randn(32, 30)
# y = np.random.randn(32, 30)

# ani = schiller_lab_tools.visualization.animate_plot(x, y)
# HTML(ani.to_jshtml())

In [ ]:
# test = np.random.randn(30, 32, 32)*10

# ani = schiller_lab_tools.visualization.animate_colormap(test)
# HTML(ani.to_jshtml())

# Microstructure analysis

In [ ]:
# k1 = schiller_lab_tools.microstructure_analysis.spherical_first_moment(phi_fill)
# D1 = 2*np.pi/k1
# D1

In [ ]:
# k2 = schiller_lab_tools.microstructure_analysis.second_moment(phi_fill)
# D2 = 2*np.pi/np.sqrt(k2)
# D2

In [ ]:
# S_i = schiller_lab_tools.microstructure_analysis.interface_order(phi_fill)
# S_i

In [ ]:
# filter_func = lambda x: np.where(x > 0, 1, 0)

# phi_bin = schiller_lab_tools.microstructure_analysis.label_regions_hk(phi_fill, filter=filter_func)

# fig, axs = plt.subplots(1, 2, figsize = (8, 4))

# slc = np.s_[:, boxDims[1]//2, :]

# ax = axs[0]
# im = ax.imshow(phi_fill[slc])
# ax.set_title("raw input")
# plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8)

# ax = axs[1]
# im = ax.imshow(phi_bin[slc])
# ax.set_title("Binarized")
# plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8)

In [ ]:
# %%time

# limit = 2/D1
# K, H, A = schiller_lab_tools.microstructure_analysis.curvature(phi_fill, (limit**2, limit), step_size = 1)

# fig, axs = plt.subplots(1, 2, figsize = (8, 4))
# a = 0.6
# b = 20

# ax = axs[0]
# ax.hist(K, alpha = a, edgecolor = "k", bins = b, density = True)
# ax.set_title("Gaussian curvature")

# ax = axs[1]
# ax.hist(H, alpha = a, edgecolor = "k", bins = b, density = True)
# ax.set_title("Mean curvature")

In [ ]:
# pn = schiller_lab_tools.microstructure_analysis.get_pn(phi_fill, parallel = {'cores':1, 'divs':[1, 1, 1], 'overlap':8})

# fig, axs = plt.subplots(2,2, figsize = (6, 6), layout = "constrained")
# a = 0.6
# b = 20

# ax = axs[0, 0]
# parameter = 'pore.equivalent_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

# ax = axs[0, 1]
# parameter = 'pore.inscribed_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

# ax = axs[1, 0]
# parameter = 'throat.equivalent_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

# ax = axs[1, 1]
# parameter = 'throat.inscribed_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

In [ ]:
# from skimage.transform import downscale_local_mean

# filter_func = lambda x: np.where(x > 0, 1, 0)
# tau_s = schiller_lab_tools.microstructure_analysis.taufactor_tortuosity(downscale_local_mean(phi_fill, (4, 4, 4)), filter = filter_func, convergence_criteria=0.1)

# Particle Analysis

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# orientations = md[['o_x', 'o_y', 'o_z']].to_numpy()
# theta, not_in_interface = schiller_lab_tools.particle_analysis.calculate_average_cos_interface_normal(phi_fill, positions, orientations, step_size = 2)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))
# a = 0.6
# b = 20
# bins, counts, patch = ax.hist(theta, alpha = a, bins = b, edgecolor = "k", density = True)
# ax.set_title("Angle to interface normal")

In [ ]:
# orientations = md[['o_z', 'o_y', 'o_x']].to_numpy()
# S = schiller_lab_tools.particle_analysis.calculate_nematic_order(orientations, director = [0, 0, 1])

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# r, gr = schiller_lab_tools.particle_analysis.calculate_rdf(boxDims, positions)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))
# ax.plot(r, gr, ls = "-", marker = "None", markerfacecolor = "None")
# ax.set_title("RDF")

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# Q4 = schiller_lab_tools.particle_analysis.calculate_ql(boxDims, positions, L = 4)
# Q6 = schiller_lab_tools.particle_analysis.calculate_ql(boxDims, positions, L = 6)

# test = np.array([Q4, Q6])
# test = test.T
# # test = whiten(test)

# clusters = 1
# centers, mean_dist = kmeans(test, k_or_guess = clusters)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))

# ax.plot(test[:, 0], test[:, 1], ls = "None", marker = ".", markerfacecolor = "None", color = "tab:blue")
# # ax.scatter(test[:, 0], test[:, 1])
# for i in range(clusters):
#     ax.plot(centers[i,0], centers[i, 1], color = "tab:red", marker = "o", ms = 10)

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# w4 = schiller_lab_tools.particle_analysis.calculate_wl(boxDims, positions, L = 4)
# w6 = schiller_lab_tools.particle_analysis.calculate_wl(boxDims, positions, L = 6)

# test = np.array([w4, w6])
# test = test.T
# # test = whiten(test)

# clusters = 1
# centers, mean_dist = kmeans(test, k_or_guess = clusters)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))
# ax.plot(test[:, 0], test[:, 1], ls = "None", marker = ".", markerfacecolor = "None")
# ax.set_xlabel("w4")
# ax.set_ylabel("w6")

# for i in range(clusters):
#     ax.plot(centers[i,0], centers[i, 1], color = "tab:red", marker = "o", ms = 10)

In [ ]:
# plt.plot(Q4, w4, ls = "None", marker = ".", color = "tab:blue", markerfacecolor = "None")
# plt.xlabel("Q6")
# plt.ylabel("w4")

In [ ]:
# 